In [4]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src/')
import wave as w
import utils as ut
import plotter
import numba_functions as nbf
import numba as nb
from copy import deepcopy
from scipy.optimize import brentq

## Plotting the speed and R0 at different D

In [31]:
def build_assay_wave(id_, r0, D, beta, gamma=1.0, cutoff=1):
    dx = ut.lin_from_two_log_points(r0, 0.5, 0.075, 20, 0.2)
    dt = ut.dt_from_cfl(0.03, D, dx)

    p = w.Vwave_pars(id_, tot_time=20000, dt=dt, dx=dx, 
                     n_x_bins=int( ut.lin_from_two_points(r0, 0.5, 100, 20, 2000)/dx ), 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**9, cutoff=cutoff,
                     t_burn=ut.lin_from_two_points(r0, 0.5, 5000, 20, 200000), 
                     t_burn_cutoff=ut.lin_from_two_points(r0, 0.5, 300, 20, 5000), 
                     back_width_fract=ut.lin_from_two_log_points(r0,  0.5, 0.2, 20, 3),
                     traj_step=int(50/dt), check_step=int(0/dt), traj_after_burn=True, verbose=True)
    return w.Vwave(p)

## FKPP regime

In [45]:
r0 = 0.5
Delta_x = 1
lambd = 100
lambda_tilde = 2*lambd/Delta_x**2

b_zero = 0.05
gamma = 0.04
D_list = np.linspace(4,6,20)*10**(-4)
b_list = (1 - lambda_tilde * D_list) * b_zero

In [ ]:
assays = []
for i in range(len(D_list)):
    assays.append(build_assay_wave(i, r0, D_list[i], b_list[i], gamma))

In [ ]:
waves = ut.multiprocess_sim(assays, 2)

In [48]:
speeds = []
for i in range(len(D_list)):
    speeds.append(np.mean(waves[i].traj.speed(100)[:]))

In [49]:
f = open('data/speed_R0_vs_D_FKPP.tsv', 'w')

header = '#D\tspeed\tR0\n'
f.write(header)

for i in range(len(D_list)):
    f.write(str(D_list[i]) + '\t')
    f.write(str(speeds[i]) + '\t')
    f.write(str(b_zero*np.exp(-lambd*D_list[i])) + '\t')
    f.write('\n')
    
f.close()


f = open('data/speed_R0_vs_D_FKPP_pars.tsv', 'w')

header = '#parameter\tvalue\n'
f.write(header)

f.write('lambda\t' + str(lambda_tilde) + '\n')
f.write('beta0\t' + str(waves[0].p.beta) + '\n')
f.write('alpha\t' + str(waves[0].p.alpha) + '\n')
f.write('gamma\t' + str(waves[0].p.gamma) + '\n')
f.write('Nh\t' + str(waves[0].p.Nh) + '\n')
f.write('M\t' + str(waves[0].p.M) + '\n')
f.write('cutoff\t' + str(waves[0].p.cutoff) + '\n')

f.close()

## linear-fitness regime

In [32]:
r0 = 22.392905694846775
D_list2 = np.linspace(1, 3, 15)*10**(-4)
b_list2 = (1 - lambda_tilde * D_list2)*b_zero

In [33]:
assays = []
for i in range(len(D_list2)):
    assays.append(build_assay_wave(i, r0, D_list2[i], b_list2[i], gamma))

In [ ]:
waves2 = ut.multiprocess_sim(assays, 2)

In [43]:
speeds2 = []
for i in range(len(D_list2)):
    speeds2.append(np.mean(waves2[i].traj.speed(50)[-200:]))

In [44]:
f = open('data/speed_R0_vs_D_linfit.tsv', 'w')

header = '#D\tspeed\tR0\n'
f.write(header)

for i in range(len(D_list2)):
    f.write(str(D_list2[i]) + '\t')
    f.write(str(speeds2[i]) + '\t')
    f.write(str(b_zero*np.exp(-lambd*D_list2[i])) + '\t')
    f.write('\n')
    
f.close()


f = open('data/speed_R0_vs_D_linfit_pars.tsv', 'w')

header = '#parameter\tvalue\n'
f.write(header)

f.write('lambda\t' + str(lambda_tilde) + '\n')
f.write('beta0\t' + str(waves2[0].p.beta) + '\n')
f.write('alpha\t' + str(waves2[0].p.alpha) + '\n')
f.write('gamma\t' + str(waves2[0].p.gamma) + '\n')
f.write('Nh\t' + str(waves2[0].p.Nh) + '\n')
f.write('M\t' + str(waves2[0].p.M) + '\n')
f.write('cutoff\t' + str(waves2[0].p.cutoff) + '\n')

f.close()